# Final Project Submission

Please fill out:
* Student name: Lewis Gathua Kamindu
* Student pace: Full time
* Scheduled project review date/time: 01/12/2023
* Instructor name: Lucille Kaleha
* Blog post URL: https://github.com/lewigi/Machine-Learning-Project-Vaccination-Patterns

# Predicting Vaccination Patterns: Analyzing Backgrounds, Opinions, and Behaviors in H1N1 and Seasonal Flu Vaccination Uptake